In [1]:
import os

import redis
import requests
from bs4 import BeautifulSoup as bs
from pprint import pprint
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)

%load_ext autoreload
%autoreload 2

# Items

In [2]:
rds = redis.StrictRedis(host='localhost', port=6379, db=0)

In [3]:
def get_page(url):
    if url in rds:
        return rds.get(url)
    else:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36'
        }

        r = requests.get(url, headers=headers)
        rds.set(url, r.text)
        return r.text

In [4]:
def get_all_cards(list_page):
    soup = bs(list_page, 'html.parser')
    return soup.select('.search-product-link')

In [5]:
def get_text_or_na(soup):
    if soup:
        return soup.get_text()
    else:
        return ''

    
def get_attr_or_na(soup, attr):
    try:
        return soup.attrs[attr]
    except AttributeError:
        return ''

    
def parse_card(card):
    import re
    is_soldout = card['data-is-soldout']
    item_id = card['data-item-id']
    product_id = card['data-product-id']
    vendor_item_id = card['data-vendor-item-id']
    href = card['href']
    name = get_text_or_na(card.select_one('.name'))
    base_price = get_text_or_na(card.select_one('.base-price'))
    discount_rate = get_text_or_na(card.select_one('.instant-discount-rate'))
    price = get_text_or_na(card.select_one('.price-value'))
    delivery_type = get_attr_or_na(card.select_one('.rocket img'), 'alt')
    tmp = get_text_or_na(card.select_one('.delivery')).strip()
    delivery_expected_date = re.sub(r'\s{2,}', ' ', tmp)
    rating = get_text_or_na(card.select_one('.star'))
    rating_count = get_text_or_na(card.select_one('.rating-total-count'))
    return {
        'is_soldout': is_soldout,
        'item_id': item_id,
        'product_id': product_id,
        'vendor_item_id': vendor_item_id,
        'href': href,
        'name': name,
        'base_price': base_price,
        'discount_rate': discount_rate,
        'price': price,
        'delivery_type': delivery_type,
        'delivery_expected_date': delivery_expected_date,
        'rating': rating,
        'rating_count': rating_count,}

In [6]:
def get_parsed_from_url(url):
    list_page = get_page(url)
    cards = get_all_cards(list_page)
    return [parse_card(c) for c in cards]

In [7]:
def get_list_page_urls():
    list_page_url_template = 'https://www.coupang.com/np/search?q=%EC%B6%94%EC%84%9D+%EC%84%A0%EB%AC%BC%EC%84%B8%ED%8A%B8&channel=auto&eventCategory=SRP&sorter=scoreDesc&listSize=36&isPriceRange=false&rating=0&page={0}'
    return [list_page_url_template.format(i) for i in range(1, 28)]

In [8]:
list_page_urls = get_list_page_urls()[:3]

while list_page_urls:
    url = list_page_urls.pop()
    print(url)
    
    parsed = get_parsed_from_url(url)
    print(parsed[0].get('product_id', ''))

https://www.coupang.com/np/search?q=%EC%B6%94%EC%84%9D+%EC%84%A0%EB%AC%BC%EC%84%B8%ED%8A%B8&channel=auto&eventCategory=SRP&sorter=scoreDesc&listSize=36&isPriceRange=false&rating=0&page=3
118610786
https://www.coupang.com/np/search?q=%EC%B6%94%EC%84%9D+%EC%84%A0%EB%AC%BC%EC%84%B8%ED%8A%B8&channel=auto&eventCategory=SRP&sorter=scoreDesc&listSize=36&isPriceRange=false&rating=0&page=2
271000832
https://www.coupang.com/np/search?q=%EC%B6%94%EC%84%9D+%EC%84%A0%EB%AC%BC%EC%84%B8%ED%8A%B8&channel=auto&eventCategory=SRP&sorter=scoreDesc&listSize=36&isPriceRange=false&rating=0&page=1
52268301


In [9]:
def get_items():
    import threading
    import time
    import pandas as pd
    import os
    if os.path.exists('items.tsv'):
        return pd.read_csv('items.tsv', sep='\t')
    
    threads = []
    result = []
    list_page_urls = get_list_page_urls()
    max_threads = min(10, len(list_page_urls))
    SLEEP_TIME = 1

    def process_list_page_urls():
        while list_page_urls:
            url = list_page_urls.pop()
            parsed = get_parsed_from_url(url)
    #         print(parsed[0].get('product_id', ''))
            result.extend(parsed)
            time.sleep(SLEEP_TIME)

    while threads or list_page_urls:
        for thread in threads:
            if not thread.is_alive():
                threads.remove(thread)
        while len(threads) < max_threads and list_page_urls:
            thread = threading.Thread(target=process_list_page_urls, daemon=True)
            thread.start()
            threads.append(thread)
        print(threads)
        for thread in threads:
            thread.join()
    
    
    items = pd.DataFrame(result)
    pd.to_csv('items.tsv', sep='\t', index=False)
    return items

In [10]:
# items.sort_values('rating', ascending=False).head(10)
# len(items)

# Add Brand Name

In [11]:
def get_url_detail_page(href):
    return 'https://www.coupang.com' + href

In [12]:
def read_or_create_file(file_name, func, read_file=True, write_file=True):
    import os
    if read_file and os.path.exists(file_name):
        return pd.read_csv(file_name, sep='\t')
    else:
        return func(file_name, write_file)

In [13]:
def add_brand_name(file_name, write_file=True):
    previous_file = 'items.tsv'
    items = pd.read_csv(previous_file, sep='\t')
    for idx, row in items.iterrows():
        href = row['href']
        url_detail_page = get_url_detail_page(href)
        detail_page = get_page(url_detail_page)
        soup = bs(detail_page, 'html.parser')
        brand_name = get_text_or_na(soup.select_one('.prod-brand-name'))
        items.loc[idx, 'brand_name'] = brand_name
    if write_file:
        items.to_csv(file_name, sep='\t', index=False)
    return item

In [14]:
# items = read_or_create_file('items_2_brand_added.tsv', add_brand_name)
# items.head()

# Breadcrumb

In [15]:
def parse_breadcrumb_link(a):
    category_href = get_attr_or_na(a, 'href')
    category_name = get_text_or_na(a).strip()
    return (category_href, category_name)


def get_breadcrumb(product_id, vendor_item_id):
    url_template_breadcrumb = 'https://www.coupang.com/vp/products/{0}/breadcrumb-gnbmenu?&invalidProduct=false&invalidUnknownProduct=false&vendorItemId={1}'
#     url_breadcrumb = 'https://www.coupang.com/vp/products/6282991/breadcrumb-gnbmenu?&invalidProduct=false&invalidUnknownProduct=false&vendorItemId=3041886650'
    url_breadcrumb = url_template_breadcrumb.format(product_id, vendor_item_id)
    breadcrumb = get_page(url_breadcrumb)
    soup = bs(breadcrumb, 'html.parser')
    links = soup.select('.breadcrumb-link')
    return [parse_breadcrumb_link(link) for link in links]

In [16]:
def add_breadcrumb(file_name, write_file=True):
    previous_file = 'items_2_brand_added.tsv'
    items = pd.read_csv(previous_file, sep='\t')
    for idx, row in items.iterrows():
        product_id = row['product_id']
        vendor_item_id = row['vendor_item_id']
        breadcrumb = get_breadcrumb(product_id, vendor_item_id)
        items.loc[idx, 'breadcrumb'] = [breadcrumb]
    if write_file:
        items.to_csv(file_name, sep='\t', index=False)
    return items

In [17]:
items = read_or_create_file('items_3_breadcrumb_added.tsv', add_breadcrumb)
items.head()

,base_price,delivery_expected_date,delivery_type,discount_rate,href,is_soldout,item_id,name,price,product_id,rating,rating_count,vendor_item_id,brand_name,breadcrumb
0,NaN,수요일 9/11 도착 예정,NaN,NaN,/vp/products/289586259?itemId=916601145&vendorItemId=5286268986,NaN,916601145,"2019년 안심특선S29호 참치 추석선물세트 가방포함, 단품","53,930",289586259,NaN,NaN,5286268986,별도표기,"[('/', '쿠팡 홈'), ('/np/categories/194276', '식품'), ('/np/categories/195443', '면/통조림/가공식품'), ('/np/categories/195484', '참치/스팸/통조림류'), ('/np/categories/195539', '통조림세트'), ('/np/categories/195543', '통조림선물세트')]"
1,NaN,수요일 9/11 도착 예정,NaN,NaN,/vp/products/289549676?itemId=916483241&vendorItemId=5286142450,NaN,916483241,"2019년 안심특선S10호 참치 추석선물세트 가방포함, 단품","49,550",289549676,NaN,NaN,5286142450,별도표기,"[('/', '쿠팡 홈'), ('/np/categories/194276', '식품'), ('/np/categories/195443', '면/통조림/가공식품'), ('/np/categories/195484', '참치/스팸/통조림류'), ('/np/categories/195539', '통조림세트'), ('/np/categories/195543', '통조림선물세트')]"
2,NaN,수요일 9/11 도착 예정,NaN,NaN,/vp/products/289747543?itemId=917123958&vendorItemId=5286844479,NaN,917123958,"2019년 안심특선S20호 참치 추석선물세트 가방포함, 단품","101,790",289747543,NaN,NaN,5286844479,별도표기,"[('/', '쿠팡 홈'), ('/np/categories/194276', '식품'), ('/np/categories/195443', '면/통조림/가공식품'), ('/np/categories/195484', '참치/스팸/통조림류'), ('/np/categories/195539', '통조림세트'), ('/np/categories/195543', '통조림선물세트')]"
3,NaN,수요일 9/11 도착 예정,NaN,NaN,/vp/products/287359636?itemId=910674417&vendorItemId=5277906533,NaN,910674417,"동원 동원참치 리챔 16호 추석 선물 세트, 단품","68,200",287359636,NaN,NaN,5277906533,동원,"[('/', '쿠팡 홈'), ('/np/categories/194276', '식품'), ('/np/categories/195443', '면/통조림/가공식품'), ('/np/categories/195484', '참치/스팸/통조림류'), ('/np/categories/195539', '통조림세트'), ('/np/categories/195543', '통조림선물세트')]"
4,NaN,9/16 도착 예정,NaN,NaN,/vp/products/277349803?itemId=878448272&vendorItemId=5243724968,NaN,878448272,"별도표기 삼육김 종합선물세트1호 추석선물 삼육김 세트, 단품","62,400",277349803,NaN,NaN,5243724968,별도표기,"[('/', '쿠팡 홈'), ('/np/categories/194276', '식품'), ('/np/categories/195443', '면/통조림/가공식품'), ('/np/categories/195484', '참치/스팸/통조림류'), ('/np/categories/195539', '통조림세트'), ('/np/categories/195543', '통조림선물세트')]"


In [18]:
items['brand_name'].value_counts()

별도표기                                                             67
청정원                                                              48
사조                                                               45
CJ제일제당                                                           43
동원                                                               41
사조해표                                                             26
히쭈유통                                                             24
[동원선물세트]                                                         24
파크몰                                                              20
Rytunyn                                                          20
카즈미                                                              17
백화점명절선물세트(식품)                                                    17
오뚜기                                                              16
뉴트리디데이                                                           15
CJ                                              

In [19]:
items['price'].str.replace(',','').astype(int).describe()
# tmp.astype(int).describe()

count    972.000000   
mean     42011.532922 
std      42650.119852 
min      750.000000   
25%      18822.500000 
50%      29800.000000 
75%      49800.000000 
max      415130.000000
Name: price, dtype: float64

# Reviews

In [20]:
def get_list_review_urls(product_id, rating_count, size=5):
    page_count = (int(rating_count) // size) + 2
    list_review_urls = []
    
    for page in range(1, page_count):
        url_review_template = 'https://www.coupang.com/vp/product/reviews?productId={0}&page={1}&size={2}&sortBy=ORDER_SCORE_ASC&ratings=&q=&viRoleCode=3&ratingSummary=true'
        url_review = url_review_template.format(product_id, page, size)
        list_review_urls.append((url_review, product_id))
    return list_review_urls

In [21]:
def get_all_review_urls():
    import re
    previous_file = 'items_3_breadcrumb_added.tsv'
    items = pd.read_csv(previous_file, sep='\t')
    all_review_urls = []
    done = []
    
    for idx, row in items.iterrows():
        product_id = row['product_id']
        tmp_rating_count = row['rating_count']
#         if pd.notna(tmp_rating_count) and (product_id not in done):
        if pd.notna(tmp_rating_count) and (product_id not in done):
            rating_count = re.search(r'\((\d+)\)', tmp_rating_count).group(1)
        else:
            continue
        done.append(product_id)
        review_urls = get_list_review_urls(product_id, rating_count)
        all_review_urls.extend(review_urls)
    return all_review_urls

In [22]:
all_review_urls = get_all_review_urls()
all_review_urls[:5]
# len(all_review_urls)

[('https://www.coupang.com/vp/product/reviews?productId=13888649&page=1&size=5&sortBy=ORDER_SCORE_ASC&ratings=&q=&viRoleCode=3&ratingSummary=true',
  13888649),
 ('https://www.coupang.com/vp/product/reviews?productId=13888649&page=2&size=5&sortBy=ORDER_SCORE_ASC&ratings=&q=&viRoleCode=3&ratingSummary=true',
  13888649),
 ('https://www.coupang.com/vp/product/reviews?productId=13888649&page=3&size=5&sortBy=ORDER_SCORE_ASC&ratings=&q=&viRoleCode=3&ratingSummary=true',
  13888649),
 ('https://www.coupang.com/vp/product/reviews?productId=13888649&page=4&size=5&sortBy=ORDER_SCORE_ASC&ratings=&q=&viRoleCode=3&ratingSummary=true',
  13888649),
 ('https://www.coupang.com/vp/product/reviews?productId=13888649&page=5&size=5&sortBy=ORDER_SCORE_ASC&ratings=&q=&viRoleCode=3&ratingSummary=true',
  13888649)]

In [23]:
def parse_article(article, product_id):
    tmp_user = article.select_one('.sdp-review__article__list__info__user__name')
    user_id = get_attr_or_na(tmp_user, 'data-member-id')
    user_name = get_text_or_na(tmp_user).strip()
    tmp_star = article.select_one('.sdp-review__article__list__info__product-info__star-orange')
    rating = get_attr_or_na(tmp_star, 'data-rating')
    tmp_review_date = article.select_one('.sdp-review__article__list__info__product-info__reg-date')
    review_date = get_text_or_na(tmp_review_date)
    tmp_product_name = article.select_one('.sdp-review__article__list__info__product-info__name')
    product_name = get_text_or_na(tmp_product_name)
    tmp_headline = article.select_one('.sdp-review__article__list__headline')
    headline = get_text_or_na(tmp_headline).strip()
    tmp_review_content = article.select_one('.sdp-review__article__list__review__content')
    review_content = get_text_or_na(tmp_review_content).strip()
    tmp_helpful_count = article.select_one('.js_reviewArticleHelpfulCount')
    helpful_count = get_text_or_na(tmp_helpful_count).strip()
    return {
        'product_id': product_id,
        'user_id': user_id,
        'user_name': user_name,
        'rating': rating,
        'review_date': review_date,
        'product_name': product_name,
        'headline': headline,
        'review_content': review_content,
        'helpful_count': helpful_count,
    }

In [24]:
def get_review(url, product_id):
    review = get_page(url)
    soup = bs(review, 'html.parser')
    articles = soup.select('article')
    return [parse_article(a, product_id) for a in articles]

In [25]:
def get_reviews_threaded(dump_file=False):
    import threading
    import time
    import pandas as pd
    import os
    if not dump_file and os.path.exists('reviews.tsv'):
        return pd.read_csv('reviews.tsv', sep='\t')
    
    threads = []
    result = []
    list_review_urls = get_all_review_urls()
    max_threads = min(10, len(list_review_urls))
    SLEEP_TIME = 1
    
    def process_list_review_urls():
        while list_review_urls:
            url, product_id = list_review_urls.pop()
            review = get_review(url, product_id)
            result.extend(review)
            time.sleep(SLEEP_TIME)
    
    while threads or list_review_urls:
        for thread in threads:
            if not thread.is_alive():
                threads.remove(thread)
        while len(threads) < max_threads and list_review_urls:
            thread = threading.Thread(target=process_list_review_urls, daemon=True)
            thread.start()
            threads.append(thread)
        print(threads)
        for thread in threads:
            thread.join()
            
    reviews = pd.DataFrame(result)
    reviews.to_csv('reviews.tsv', sep='\t', index=False)
    return reviews

In [26]:
reviews = get_reviews_threaded()
print(reviews.shape)
reviews.head()

(57446, 9)


,headline,helpful_count,product_id,product_name,rating,review_content,review_date,user_id,user_name
0,좋아요,NaN,37854418,"샘표 진심담은 S호 선물세트, 1세트",5,NaN,2019.09.03,120017854,L*a
1,NaN,0,37854418,"샘표 진심담은 S호 선물세트, 1세트",5,구성최고입니다,2019.09.01,8718940,기*정
2,NaN,0,37854418,"샘표 진심담은 S호 선물세트, 1세트",5,상품구성이 알찹니다 많이 파세요. 큰 상자에 상품이 한 개씩 들어 있어요 과대포장 너무 심합니다 세 개 중 한 개 배송이 안 돼서 추가로 신청했습니다 다,2019.09.06,13478027,신*자
3,추천합니다,0,35815896,"넛츠앤 호두 220g + 카카오닙스 300g + 건크랜베리 300g 오너 견과 선물세트 1호 + 쇼핑백, 820g, 1세트",5,선물했어요,2018.09.15,273903,안*균
4,NaN,0,271567746,"청정원 7호 선물세트, 1세트",5,"하루만에 도착한 로켓배송 좋아요\n올리고당,카놀라유부터 침치, 스팸, 홍초까지 알찬 구성 좋아요\n깔끔한 비닐 포장도에 들구 다니기 편하게 손잡이까지 좋아요\n이번 추석은 청정원과 함께~!",2019.08.30,120077320,신*섭


In [27]:
reviews.sort_values('helpful_count', ascending=False).head()

,headline,helpful_count,product_id,product_name,rating,review_content,review_date,user_id,user_name
9053,저렴한 가격에 잘삿네요,98,2798778,"6년근 고려홍삼정 365 4개입 + 쇼핑백, 960g, 1세트",4,연락도없이 문자로만주시고 경비실에 남기면..그 무거운6개셋트를 구매햇는데...ㅠㅜ\n6 ×4=24병...승강기교체로 계단 들고 8층까지 올라가느라 개고생함...\n병이 깨졋다는 소리듣고 하나하나 뜯어보앗음..\n선물용으로 구매햇는데 병마다 벗겨짐이 눈에보이고 뚜껑도 벗겨짐이 눈에보여 실망..24병 모두 벗겨짐이 다 있음.\n케이스도 벗겨짐이 넘심함....병포장상태를 더 신경썻으면 싶음..,2017.05.07,108428921,김*숙
57442,NaN,96,13888649,"담터 호두 아몬드 율무차 115포 선물세트, 2070g, 1세트",2,솔직히 화가나네요 전 담티 이제품 몇년째 먹고있는 사람으로 오프라인에서 도매가로도 이가격엔 못살거같아 구매했는데 첫봉을 타서 먹는순간 이건 아니다라는 생각에 다시 겉상자 확인까지ㅠ\n기획으로 구성하다보니 내용물이 부실하더군요\n원래한개씩타 마시지만 이번껀 싱거워 2커씩타마십니다 \n제입맛이 변한건가요?ㅠ,2016.03.14,100307796,강*명
44009,그냥 그저그래요. 끝맛이달게느껴지네요.,95,185907601,"비타할로 진심을 담은 프리미엄 석류즙 100%, 70ml, 30개입",2,"지금 1년가까이 석류를 꾸준히 섭취하고있어요.\n\n직접석류를 갈아서 먹기도하고, 시중 포장된석류도 구매해서\n\n마셔봤지만, 이제품은 좀 그닥 그저그래요.\n\n원래,끝맛이 신맛이 나야 제대로 성분이 들어간거인데,\n\n이건 색상도 너무 묽거나, 끝맛이 단맛이나네요.\n\n양도 시중 석류음료중, 조금 양도 덜합니다.\n\n그냥 그저그러네요.",2019.03.19,10318461,정*원
4531,홍삼스틱으로 가격대비 괜츈합니당~☆,91,4036698,"정원삼 6년근 고려홍삼정 365 스틱 30포 + 쇼핑백, 300g, 1세트",5,"요즘 제가 삶이 고단한지;;;;;;ㅎㅎ\n면역력도 떨어지고 피곤해서 신랑이 홍삼스틱을 사서 먹으라했는데\n정관장꺼는 너무 비싸고 효과도 없음;;\n돈아까울까봐 가성비좋은것들중 후기검색해 쿠팡에서 구매했어욤\n한포먹었는데 와~엄청 진하네요\n물탄것 같지않은~♡\n쓴거 못먹는 애기입맛인데 건강에 좋다생각하고 먹으니 먹어지네요\n그리고 귤하나 먹었어요 너무 진해서\n홍삼 자주 먹었던 신랑도 먹어보더니 잘샀다고 하네요\n재구매의사있어요♡♡♡♡♡\n아참 한포의 양이 작은건 살짝 슬퍼요ㅋㅋ\n\n+) 전 24,900원에 구매했어요 참고하세용~\n✏2016년12월27일구매",2016.12.28,12783060,권미란다
17927,"보들보들.폭신폭신한식감의""마드레드""우유랑함께해요",90,34253432,신라명과 [본사배송] 마드레느(대) 선물특가 마들렌,5,"♠ 상품평 읽을시,사진.글과 함께하세요 ♠\n\n\n안녕하세요 : )\n제가 좋아하는, 아이들도 좋아하는 간식중 하나 소개 합니다. 신라명과 제품으로 유명합니다. 빵집마다 다 있는 메뉴예요. 갑자기 먹고싶은데 빵집은 너무 가격이 도도해서 쿠팡서 지대로 된 신랑명과 제품으로 구입해 아이들이랑 만찬을 즐겼어요^^*\n\n\n⚅상품상세 정보\n\n♬ 제품명 : 신라명과 마드레느 (대)\n\n♬ 용량 : 15g × 32개입 (총 480g)\n\n♬ 생산일 : 출고 당일\n\n♬ 유통기한 : 제조년일로부터 6일 (17. 01.19)\n\n♬ 구성품 : 선물쇼핑백, 본상품 \n\n♬ 1회제공량 : 5개 (285kcal)\n\n♬ 주문일 : 17. 01. 10 업체배송으로 주문\n\n♬ 배송일 : 17. 01. 12 타택배사로 배송\n\n♬ 포장과 배송 : 업체 배송이지만 포장도 깔끔 하며, 배송날짜보다 빠른 배송이 이뤄져 기분이 좋았어요\n\n♬ 만족도 ⭐⭐⭐⭐⭐\n\n\n⚄ 상품의 특장점\n\n▶ 프리미엄 디저트 작은 스틱형으로 먹기에 편함\n\n▶ 부드럽고 촉촉한 맛이 일품인 프랑스 전통 간식\n\n▶ 버터의 품미를 담은 촉촉한 미니케이크로 커피와 우유랑 조화로움\n\n▶ 하나씩 낱개 포장되어 간편하게 맛 즐기기좋음\n\n▶ 작은 사이즈로 휴대하기도 편리해 언제 어디서든 간식을 즐길 수 있음\n\n▶ 깔끔한 상자 포장 으로 선물용으로 추천함\n\n▶ 마드레느에는 식품보존제를 넣지않아 생산일부터 6일까지 다소 유통기한이 짧음\n\n▶ 철저한 위생관리속에서 꼼꼼하게 만듬\n- 2005년 제과 업계 최초 HACCP인증 받음\n\n\n⚂ 시식 후, 개인적인 의견\n\n빵집의 마드레느보다 큰사이즈의 스틱형으로 아담해서 먹기 편하고 위생적이라 좋아요.\n달콤함과 촉촉함이 모두 만족시켜 주는 ""마드레느"" 언제 어디서든 맛있게 즐길 수 있는 ""마드레느"" 남녀노소 누구나 좋아할 맛을 소유한 최강자 !\n""신라명과""에서 정성으로 만든 것 같은 ""마드레느""의 깊은 맛은 까닭이 있었어요. 고급버터와 계란(요즘은, 금란이죠)을 넣어 부드러운 버터향과 은은하게 느껴지는 달콤함은 예술이죠^^*\n\n낱개포장으로 32개의 구성되어 촉촉함이 오래오래 유지가능 하기때문입니다. \n휴대하기 편해 어디서든 간편하게 누구나 드실 수 있는 고급진 간식이라지요~^^*\n\n아이들에게는 부드러운 우유와 어른들은 취향껏 따뜻한 커피로 마드레느의 깊은맛을 더해주어 더 부드럽고, 더 달콤한 맛있는 식감을 느낄 수 있어 너무 쪼아요~♡\n\n나른하고 지친오후, 마드레느의 부드럽고 달콤한에 취해보세요! 그래서 기운내셔서 화이팅하세요~!!\n피곤하고 지친하루에 ""마드레느""와 함께여도 괜찮으실거라 생각돼요. 저도 오후에 달달하고, 부드러운게 먹고싶어지던데~ 오늘도 8개정도는 먹은 것 같아요. 전 우유나 두유랑 같이 먹으니 샤르르~ 녹아 입안에서부터 마드레느는 없어져 안먹은 것 같지만, 점점 불러오는 배를 보며 뒤늦은 후회는 소용이 없다지요ㅠㅠ 1회제공량 5개에 285kcal.\n만만하지않는 칼로리예요ㅠㅠ 내일부터는 조심해서 먹어야지 ! 라고, 또 다짐해 봅니다 ㅡㅡ;;\n곧 무너질 다짐이지만요 ㅠㅠ\n\n쇼핑백까지 함께 배송이 되어서 선물용으로도 추천드리고 싶으네요. 고급진 포장에 신라명과라고 찍힌 쇼핑백에 간단한 메모와 넣어 선물해 드리면 어느 선물에 비해 손색이 없을 듯 싶으네요~^^\n\n\n\n⚅ 마드레느 보관법\n\n마드레느는 냉장 보관 마시고, ""냉동보관""하세요!\n살아있는 빵은 만든 후부터 서서히 노화가 진행됩니다\n냉동보관을 하게되면 노화진행이 급격히 느려지고 빵의 본연의 맛이 오래갑니다. 드실 때에는 실온에서 천천히 해동하시면 고유의 맛을 느끼실 수 있습니다. 꼭! 냉동보관하세요! ⛤\n\n\n지인들에게 음력설에 부담없는\n선물으로도 추천해요~♡♡\n\n\n제 상품평에 찾아주신님,\n진심으로 감사드립니다~♡\n\nㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ",2017.01.12,101011711,봄날은지금


In [28]:
reviews['product_id'].nunique()

349

# Reviews per User

In [29]:
def get_all_url_users_review_meta(reviews):
    url_template_users_review_meta = 'https://www.coupang.com/vp/product/reviews/profile/{0}'
    users = set(reviews['user_id'].tolist())
    return [(url_template_users_review_meta.format(uid), uid) for uid in users]

In [30]:
def parse_meta_html(html, uid):
    meta = bs(html)
    user_name = get_text_or_na(meta.select_one('.sdp-review__profile__article__info__name'))
    helpful_count = get_text_or_na(meta.select_one('.sdp-review__profile__article__info__helpful__count'))
    write_count = get_text_or_na(meta.select_one('.sdp-review__profile__article__info__write__count'))
    return {
        'uid': uid,
        'user_name': user_name,
        'helpful_count': helpful_count,
        'write_count': write_count}

In [31]:
import asyncio
from aiohttp import ClientSession

async def get_users_review_meta_parsed(url_meta, uid, sema):
    if url_meta in rds:
        html = rds.get(url_meta)
    else:
        headers = {
            'Accept-Language': "en,en-US;q=0.9,ko;q=0.8",
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36'
        }
        async with sema, ClientSession(headers=headers) as session, session.get(url_meta) as resp:
            await asyncio.sleep(0.5)
            html = await resp.text()
            if resp.status == 200:
                rds.set(url_meta, html)
            else:
                print(f'{url_meta} failed.')
    return parse_meta_html(html, uid)

In [64]:
headers = {
    'Accept-Language': "en,en-US;q=0.9,ko;q=0.8",
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36'
}
r = requests.get('https://www.coupang.com', headers=headers)
r.text

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n<!doctype html>\n<!--[if lte IE 7 ]><html class="lt-ie9 lt-ie8" lang="ko-KR"><![endif]-->\n<!--[if IE 8 ]><html class="lt-ie9" lang="ko-KR"><![endif]-->\n<!--[if (gte IE 9)|!(IE)]><!--><html lang="ko-KR"><!--<![endif]-->\n<head>\n    <meta charset="utf-8" />\n    <meta name="google-site-verification" content="zaNrGtrOLMjglkziY2IvmL8dOXyCWHGArDHqFazJQVI" />\n    <meta http-equiv="x-dns-prefetch-control" content="on" />\n    <link rel="dns-prefetch" href="//cart.coupang.com" />\n    <link rel="dns-prefetch" href="//assets.coupang.com" />\n    <link rel="dns-prefetch" href="//assets2.coupang.com" />\n    <link rel="dns-prefetch" href="//assets.coupangcdn.com" />\n    <link rel="dns-prefetch" href="//asset1.coupangcdn.com" />\n    <link rel="dns-prefetch" href="//private.coupang.com" />\n    <link rel="dns-prefetch" href="//triforce.coupang.com" />\n    <link rel="dns-prefetch" href="//img1a.coupangcdn.com" />\n    <link rel="dns-prefetch" href="//image1.co

In [47]:
async def get_users_review_meta_df(file_name, write_file=True):
    all_url_meta = get_all_url_users_review_meta(reviews)
    sema = asyncio.BoundedSemaphore(10)  # limit maximum number of connection to 10
    tasks = (asyncio.create_task(get_users_review_meta_parsed(url_meta[0], url_meta[1], sema))
             for url_meta in all_url_meta)
    parsed_list = await asyncio.gather(*tasks)
    df_users_review_meta = pd.DataFrame(parsed_list)
    if write_file:
        df_users_review_meta.to_csv(file_name, sep='\t', index=False)
    return df_users_review_meta

In [33]:
file_name = 'users_review_meta.tsv'
if os.path.exists(file_name):
    meta = pd.read_csv(file_name, sep='\t')
else:
    meta = await asyncio.create_task(get_users_review_meta_df(file_name, write_file=False))

meta.head()

,uid,user_name,helpful_count,write_count
0,14417920,김*호,40,291
1,37617664,김*영,17,62
2,22413314,허*화,0,11
3,109314052,김*현,96,282
4,25690116,김*지,0,2


In [34]:
len(meta)

47188

In [35]:
meta.sort_values('helpful_count', ascending=False).head(10)

,uid,user_name,helpful_count,write_count
31262,101011711,봄날은지금,48134,2574
18384,24167508,윤선미,24546,1175
36063,19760447,조혜정,20190,2465
33624,31943265,즐거운소비,18348,1633
3209,14950982,착한엄마,17152,1419
6456,13386871,SHINE0507,16914,975
25420,107942145,솔직해라,16886,914
14150,101095169,초코엄마,15070,1685
33654,100756128,호동새댁,14644,1805
26084,108468368,주부구매자,13487,1755


In [36]:
meta.sort_values('write_count', ascending=False).head(10)

,uid,user_name,helpful_count,write_count
31360,18567667,윤*아,40,3728
30393,39012156,이*희,100,3512
47133,3014519,김진희,6739,3399
3647,2500258,최*서,157,3107
43858,62250168,강*하,39,3061
35241,100629454,정*지,5,3004
44749,23193223,최고의쇼핑,12329,2973
13615,34378108,오*민,139,2859
36586,107973374,추*경,89,2829
42054,30787738,김*아,66,2633


In [56]:
items.sample(5)

,base_price,delivery_expected_date,delivery_type,discount_rate,href,is_soldout,item_id,name,price,product_id,rating,rating_count,vendor_item_id,brand_name,breadcrumb
659,"12,900",모레(토) 9/7 도착 예정,NaN,24%,/vp/products/37611807?itemId=138559639&vendorItemId=3010649062,NaN,138559639,"라바티 허브차 선물세트/추석선물세트, 미니병3종세트, 미니병3개, 1개","9,700",37611807,4.5,(17),3010649062,라바티,"[('/', '쿠팡 홈'), ('/np/categories/194276', '식품'), ('/np/categories/196393', '선물세트관'), ('/np/categories/196423', '커피/차/음료'), ('/np/categories/196426', '차 선물세트')]"
503,NaN,수요일 9/11 도착 예정,NaN,NaN,/vp/products/294741301?itemId=929821045&vendorItemId=5307354428,NaN,929821045,"2019 사조 추석선물세트 사조안심 S2호, 단품","42,000",294741301,NaN,NaN,5307354428,사조,"[('/', '쿠팡 홈'), ('/np/categories/194276', '식품'), ('/np/categories/195443', '면/통조림/가공식품'), ('/np/categories/195484', '참치/스팸/통조림류'), ('/np/categories/195539', '통조림세트'), ('/np/categories/195543', '통조림선물세트')]"
540,NaN,화요일 9/10 도착 예정,NaN,NaN,/vp/products/72533437?itemId=241830332&vendorItemId=3595497143,NaN,241830332,"설탕이 아닌 벌꿀로 만든 제주자연 꼬마3종세트 | 시원한 탄산수에 넣어 에이드로 즐겨보세요~, 1개, 02.미니 꿀차 2종세트+쇼핑백증정","7,200",72533437,NaN,NaN,3595497143,설탕이 아닌 벌꿀로 만든,"[('/', '쿠팡 홈'), ('/np/categories/194276', '식품'), ('/np/categories/196393', '선물세트관'), ('/np/categories/196423', '커피/차/음료'), ('/np/categories/196426', '차 선물세트')]"
667,"34,000",내일(금) 9/6 도착 보장,로켓배송,47%,/vp/products/18244118?itemId=73485139&vendorItemId=3120388788,NaN,73485139,"슈퍼잼 슈퍼너츠 피넛버터 스무스 + 크런키 2종 선물세트, 600g, 1세트","17,920",18244118,5.0,(57),3120388788,슈퍼잼,"[('/', '쿠팡 홈'), ('/np/categories/194276', '식품'), ('/np/categories/195443', '면/통조림/가공식품'), ('/np/categories/195484', '참치/스팸/통조림류'), ('/np/categories/195528', '잼/스프레드'), ('/np/categories/195535', '잼 선물세트')]"
781,"19,900",내일(금) 9/6 도착 보장,로켓배송,15%,/vp/products/51770016?itemId=97773978&vendorItemId=4523981726,NaN,97773978,"고려인삼유통 홍삼 가득 담은 6년근 골드스틱 + 쇼핑백, 360g, 1세트","16,900",51770016,4.0,(3539),4523981726,고려인삼유통,"[('/', '쿠팡 홈'), ('/np/categories/305798', '헬스/건강식품'), ('/np/categories/311209', '홍삼/인삼'), ('/np/categories/311211', '홍삼진액/파우치')]"


In [62]:
reviews.sample(5)

,headline,helpful_count,product_id,product_name,rating,review_content,review_date,user_id,user_name
45785,NaN,0,2534209,"투데이넛 너트한줌 프리미엄 실속형 50개입, 1000g, 1세트",5,바쁜아침에 이거 한봉지에 두유한개 마시고 출근하는데\n좋네요,2019.01.03,13542622,김*윤
111,NaN,0,2534211,"투데이넛 너트한줌 요거트 실속형, 20g, 50봉",5,가격대는 좀비싼거같은데 너무맛있어서 자주시켜먹어요 \n돈아깝다는생각이 안들어요 ㅎㅎ,2017.12.26,109316489,최*식
23412,NaN,NaN,35632925,"락토핏 생유산균 골드 선물세트, 300g, 1세트",4,NaN,2019.02.15,106738805,윤*희
17691,NaN,0,34253432,신라명과 [본사배송] 마드레느(대) 선물특가 마들렌,2,선물용으로 샀는데..부피가 작아서 민망했네요..\n맛은..먹어보질 못해서..,2017.01.22,2946364,이*우
37438,완전 맘에들어요.,0,3562321,"넛츠팜 프리미엄 4종 견과세트 5호, 750g, 1세트",5,답례품으로 구입했는데 고급스럽고 좋았어요.\n가성비 갑인듯합니다,2018.06.04,251727,유*란


In [38]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)